In [18]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTENC
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

import matplotlib.pyplot as plt
import warnings

from sklearn.model_selection import cross_val_score,KFold
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import (
    make_scorer, 
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score,
    confusion_matrix)
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

In [3]:
# 函数列表

def oversampling(data):
    Y = data["TARGET"]
    X = data.loc[:, data.columns != "TARGET"]
    cata_col = X.select_dtypes(include=object).columns
    cata_col_indices = [X.columns.get_loc(col) for col in cata_col]
    sm = SMOTENC(random_state=1, categorical_features=cata_col_indices, k_neighbors=3)
    X_res, y_res = sm.fit_resample(X, Y)

    ## array->df
    X = pd.DataFrame(X_res)
    # get new index
    origin_ID = X_res.index.to_list()
    num_respl = X.shape[0]-len(origin_ID)
    resample_ID = [i for i in range(origin_ID[-1]+1,origin_ID[-1]+num_respl+1)]
    origin_ID.extend(resample_ID) #combine origin and resample id

    # combine X、y和index
    resampled_data = pd.concat([pd.DataFrame(X_res), pd.Series(y_res, name='TARGET')], axis=1)
    resampled_data = pd.concat([pd.Series(origin_ID, name="ID"),resampled_data],axis=1)
    resampled_data.set_index("ID", inplace=True)

    return resampled_data
    

def undersampling(data):
    # Get the target variable and features
    Y = data["TARGET"]
    X = data.drop(columns=["TARGET"])
    
    # Create a RandomUnderSampler object
    rus = RandomUnderSampler(random_state=1)
    X_res, y_res = rus.fit_resample(X, Y)
    
    # Create a new DataFrame to store the resampled data
    # Use a dictionary to build the DataFrame and ensure the indices are from the original dataset
    resampled_data = pd.concat([pd.DataFrame(X_res, columns=X.columns), pd.Series(y_res, name='TARGET')], axis=1)
    
    # Select the corresponding indices from the original dataset
    original_index = data.iloc[rus.sample_indices_].index
    
    # Set the indices of the resampled data to the original indices
    resampled_data.set_index(original_index, inplace=True)
    
    return resampled_data


def encode_cata_data(train_data):
    
    # Initialize empty DataFrames to hold the encoded training and testing data
    encoder_train = pd.DataFrame(index = train_data.index)
    #encoder_test = pd.DataFrame(index = test_data.index)
    selected_features = train_data.select_dtypes(include=['object'])
    # Initialize the encoders
    one_hot_encoder = OneHotEncoder(sparse_output=False)
    label_encoder = LabelEncoder()
    
    for column in selected_features.columns:
        unique_values = selected_features[column].nunique()
        
        if unique_values == 2:  # For two-class features
            encoder_train[column] = label_encoder.fit_transform(selected_features[column])
            #encoder_test[column] = label_encoder.transform(test_data[column])
        else:  # For multi-class features
            # Fit the OneHotEncoder on the training data for the current column
            one_hot_encoder.fit(selected_features[[column]])  
            
            ohe_encoded_train = one_hot_encoder.transform(selected_features[[column]])  # Transform the training data using the fitted OneHotEncoder
            ohe_encoded_train = pd.DataFrame(ohe_encoded_train, columns=one_hot_encoder.get_feature_names_out([column]),index = selected_features.index)
            encoder_train = pd.concat([encoder_train, ohe_encoded_train], axis=1)

    encoder_train = pd.concat([encoder_train, train_data.select_dtypes(include=['number'])], axis=1)
            
    return encoder_train


In [2]:
# 读取 train_Data 文件
application_train = pd.read_parquet('/Users/xiaohehe/Desktop/hkust/24-25fall/fintech/project1/data/combine_data.parquet')
#application_test = pd.read_csv('/Users/xiaohehe/Desktop/hkust/24-25fall/fintech/project1/data/application_test.csv')

# 打印 DataFrame
application_train

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,SK_ID_CURR_var_credit_goods_diff_last_5,SK_ID_CURR_var_credit_goods_ratio_last_5,SK_ID_CURR_var_application_credit_diff_last_5,SK_ID_CURR_var_application_credit_ratio_last_5,previous_application_number_of_prev_application,previous_application_fraction_of_refused_applications,exc_tf_1,exc_tf_2,exc_tf_3,TARGET
0,100002,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,...,1.785162e+09,0.017306,4.953836e+09,0.069064,1.0,0.000000,0,0,0,1
1,100003,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,...,5.728765e+09,0.007015,5.728765e+09,0.005324,3.0,0.000000,0,0,0,0
2,100004,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,...,1.785162e+09,0.017306,4.953836e+09,0.069064,1.0,0.000000,0,0,0,0
3,100006,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,...,2.378708e+10,0.050180,9.514831e+09,0.028940,9.0,0.111111,0,0,0,0
4,100007,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,...,6.140349e+08,0.012347,6.140349e+08,0.007897,6.0,0.000000,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,456221,Cash loans,F,N,Y,0,121500.0,412560.0,17473.5,270000.0,...,1.876021e+09,0.016400,4.470164e+09,0.065782,1.0,0.000000,0,0,0,2
48740,456222,Cash loans,F,N,N,2,157500.0,622413.0,31909.5,495000.0,...,1.640626e+08,0.012453,1.640626e+08,0.009809,4.0,0.000000,0,0,0,2
48741,456223,Cash loans,F,Y,Y,1,202500.0,315000.0,33205.5,315000.0,...,1.459742e+08,0.005002,1.459742e+08,0.006176,2.0,0.000000,0,0,0,2
48742,456224,Cash loans,M,N,N,0,225000.0,450000.0,25128.0,450000.0,...,5.884195e+08,0.010553,5.010032e+08,0.007871,5.0,0.400000,0,0,0,2


In [4]:
application_train = encode_cata_data(application_train)
application_train

,NAME_CONTRACT_TYPE,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,...,SK_ID_CURR_var_credit_goods_diff_last_5,SK_ID_CURR_var_credit_goods_ratio_last_5,SK_ID_CURR_var_application_credit_diff_last_5,SK_ID_CURR_var_application_credit_ratio_last_5,previous_application_number_of_prev_application,previous_application_fraction_of_refused_applications,exc_tf_1,exc_tf_2,exc_tf_3,TARGET
0,0,0.0,1.0,0.0,0,1,0.0,0.0,0.0,0.0,...,1.785162e+09,0.017306,4.953836e+09,0.069064,1.0,0.000000,0,0,0,1
1,0,1.0,0.0,0.0,0,0,0.0,1.0,0.0,0.0,...,5.728765e+09,0.007015,5.728765e+09,0.005324,3.0,0.000000,0,0,0,0
2,1,0.0,1.0,0.0,1,1,0.0,0.0,0.0,0.0,...,1.785162e+09,0.017306,4.953836e+09,0.069064,1.0,0.000000,0,0,0,0
3,0,1.0,0.0,0.0,0,1,0.0,0.0,0.0,0.0,...,2.378708e+10,0.050180,9.514831e+09,0.028940,9.0,0.111111,0,0,0,0
4,0,0.0,1.0,0.0,0,1,0.0,0.0,0.0,0.0,...,6.140349e+08,0.012347,6.140349e+08,0.007897,6.0,0.000000,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,0,1.0,0.0,0.0,0,1,0.0,0.0,0.0,0.0,...,1.876021e+09,0.016400,4.470164e+09,0.065782,1.0,0.000000,0,0,0,2
48740,0,1.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,...,1.640626e+08,0.012453,1.640626e+08,0.009809,4.0,0.000000,0,0,0,2
48741,0,1.0,0.0,0.0,1,1,0.0,0.0,0.0,0.0,...,1.459742e+08,0.005002,1.459742e+08,0.006176,2.0,0.000000,0,0,0,2
48742,0,0.0,1.0,0.0,0,0,0.0,1.0,0.0,0.0,...,5.884195e+08,0.010553,5.010032e+08,0.007871,5.0,0.400000,0,0,0,2


In [6]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(train_data.drop('TARGET',axis = 1))

# 使用训练集的均值和方差标准化测试集
X_test_scaled = scaler.transform(test_data)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=train_data.drop('TARGET',axis = 1).columns, index=train_data.index)
train_data[X_train_scaled.columns] = X_train_scaled
test_data = pd.DataFrame(X_test_scaled, columns=test_data.columns, index=test_data.index)


/var/folders/sj/_sb0907j00b8ddbhfpnj9q0m0000gn/T/ipykernel_95512/263061316.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data[X_train_scaled.columns] = X_train_scaled


In [ ]:
# 删除共线性较高的因子
corr = pd.read_parquet('/home/cheam/fin_project1/fintech_pro1_data/corr.parquet')
sorted_result = pd.read_parquet('/home/cheam/fin_project1/fintech_pro1_data/sorted_results.parquet')

corr_matrix = corr.abs()  # 取绝对值

# 创建一个用于存储要删除的特征的集合
to_drop = set()

# 遍历相关性矩阵
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if corr_matrix.iloc[i, j] > 0.95:  # 如果相关性大于 0.95
            colname = corr_matrix.columns[i]  # 获取列名
            to_drop.add(colname)  # 添加到删除集合

# 删除特征
df_reduced = application_train_resample.drop(columns=to_drop)

print("Remaining features:", df_reduced.columns.tolist())

In [5]:
to_drop ={
 'CODE_GENDER_NAME_EDUCATION_TYPE_max_AMT_ANNUITY_abs_diff',
 'CODE_GENDER_NAME_EDUCATION_TYPE_max_AMT_CREDIT_abs_diff',
 'CODE_GENDER_NAME_EDUCATION_TYPE_max_OWN_CAR_AGE_abs_diff',
 'CODE_GENDER_NAME_EDUCATION_TYPE_mean_EXT_SOURCE_2_diff',
 'CODE_GENDER_ORGANIZATION_TYPE_mean_AMT_ANNUITY_diff',
 'CODE_GENDER_ORGANIZATION_TYPE_mean_AMT_INCOME_TOTAL_abs_diff',
 'CODE_GENDER_ORGANIZATION_TYPE_mean_AMT_INCOME_TOTAL_diff',
 'CODE_GENDER_ORGANIZATION_TYPE_mean_DAYS_REGISTRATION_diff',
 'CODE_GENDER_REG_CITY_NOT_WORK_CITY_mean_AMT_ANNUITY_abs_diff',
 'CODE_GENDER_REG_CITY_NOT_WORK_CITY_mean_AMT_ANNUITY_diff',
 'CODE_GENDER_REG_CITY_NOT_WORK_CITY_mean_CNT_CHILDREN_diff',
 'CODE_GENDER_REG_CITY_NOT_WORK_CITY_mean_DAYS_ID_PUBLISH_diff',
 'NAME_EDUCATION_TYPE_OCCUPATION_TYPE_REG_CITY_NOT_WORK_CITY_mean_ELEVATORS_AVG_diff',
 'NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_AMT_CREDIT_diff',
 'NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_AMT_REQ_CREDIT_BUREAU_YEAR_diff',
 'NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_APARTMENTS_AVG_diff',
 'NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_BASEMENTAREA_AVG_diff',
 'NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_NONLIVINGAREA_AVG_diff',
 'NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_OWN_CAR_AGE_diff',
 'NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_YEARS_BUILD_AVG_diff',
 'NONLIVINGAPARTMENTS_MEDI',
 'NONLIVINGAPARTMENTS_MODE',
 'NONLIVINGAREA_MEDI',
 'NONLIVINGAREA_MODE',
 'OBS_60_CNT_SOCIAL_CIRCLE',
 'OCCUPATION_TYPE_mean_AMT_ANNUITY_abs_diff',
 'OCCUPATION_TYPE_mean_AMT_ANNUITY_diff',
 'OCCUPATION_TYPE_mean_CNT_CHILDREN_abs_diff',
 'OCCUPATION_TYPE_mean_CNT_CHILDREN_diff',
 'OCCUPATION_TYPE_mean_CNT_FAM_MEMBERS_diff',
 'OCCUPATION_TYPE_mean_DAYS_BIRTH_diff',
 'OCCUPATION_TYPE_mean_DAYS_EMPLOYED_diff',
 'OCCUPATION_TYPE_mean_DAYS_ID_PUBLISH_diff',
 'OCCUPATION_TYPE_mean_DAYS_REGISTRATION_abs_diff',
 'OCCUPATION_TYPE_mean_DAYS_REGISTRATION_diff',
 'REGION_RATING_CLIENT_W_CITY',
 'SK_ID_CURR_count_pos_cash_paid_late_with_tolerance_last_3',
 'SK_ID_CURR_count_pos_cash_paid_late_with_tolerance_last_5',
 'SK_ID_CURR_max_AMT_ANNUITY_last_1_x',
 'SK_ID_CURR_max_AMT_ANNUITY_last_1_y',
 'SK_ID_CURR_max_AMT_APPLICATION_last_1',
 'SK_ID_CURR_max_AMT_CREDIT_MAX_OVERDUE_last_1',
 'SK_ID_CURR_max_AMT_CREDIT_MAX_OVERDUE_last_3',
 'SK_ID_CURR_max_AMT_CREDIT_MAX_OVERDUE_last_5',
 'SK_ID_CURR_max_AMT_CREDIT_SUM_DEBT_last_1',
 'SK_ID_CURR_max_AMT_CREDIT_SUM_LIMIT_last_1',
 'SK_ID_CURR_max_AMT_CREDIT_SUM_OVERDUE_last_1',
 'SK_ID_CURR_max_AMT_CREDIT_SUM_OVERDUE_last_5',
 'SK_ID_CURR_max_AMT_CREDIT_SUM_last_1',
 'SK_ID_CURR_max_AMT_CREDIT_last_1',
 'SK_ID_CURR_max_AMT_CREDIT_last_3',
 'SK_ID_CURR_max_AMT_CREDIT_last_5',
 'SK_ID_CURR_max_AMT_DOWN_PAYMENT_last_1',
 'SK_ID_CURR_max_AMT_GOODS_PRICE_last_1',
 'SK_ID_CURR_max_AMT_GOODS_PRICE_last_3',
 'SK_ID_CURR_max_AMT_GOODS_PRICE_last_5',
 'SK_ID_CURR_max_AMT_INSTALMENT_last_1',
 'SK_ID_CURR_max_AMT_INSTALMENT_last_3',
 'SK_ID_CURR_max_AMT_PAYMENT_last_1',
 'SK_ID_CURR_max_AMT_PAYMENT_last_3',
 'SK_ID_CURR_max_AMT_PAYMENT_last_5',
 'SK_ID_CURR_max_CNT_CREDIT_PROLONG_last_1',
 'SK_ID_CURR_max_CNT_INSTALMENT_MATURE_CUM',
 'SK_ID_CURR_max_CNT_PAYMENT_last_1',
 'SK_ID_CURR_max_CREDIT_DAY_OVERDUE_last_1',
 'SK_ID_CURR_max_DAYS_CREDIT_ENDDATE_last_1',
 'SK_ID_CURR_max_DAYS_CREDIT_UPDATE_last_1',
 'SK_ID_CURR_max_DAYS_CREDIT_UPDATE_last_5',
 'SK_ID_CURR_max_DAYS_CREDIT_last_1',
 'SK_ID_CURR_max_DAYS_CREDIT_last_3',
 'SK_ID_CURR_max_DAYS_CREDIT_last_5',
 'SK_ID_CURR_max_DAYS_DECISION_last_1',
 'SK_ID_CURR_max_DAYS_DECISION_last_3',
 'SK_ID_CURR_max_DAYS_DECISION_last_5',
 'SK_ID_CURR_max_DAYS_ENTRY_PAYMENT_last_1',
 'SK_ID_CURR_max_DAYS_ENTRY_PAYMENT_last_3',
 'SK_ID_CURR_max_DAYS_ENTRY_PAYMENT_last_5',
 'SK_ID_CURR_max_DAYS_FIRST_DRAWING_last_1',
 'SK_ID_CURR_max_DAYS_INSTALMENT_last_1',
 'SK_ID_CURR_max_DAYS_INSTALMENT_last_3',
 'SK_ID_CURR_max_DAYS_INSTALMENT_last_5',
 'SK_ID_CURR_max_HOUR_APPR_PROCESS_START_last_1',
 'SK_ID_CURR_max_NAME_PAYMENT_TYPE_account_last_1',
 'SK_ID_CURR_max_NAME_PAYMENT_TYPE_bank_last_1',
 'SK_ID_CURR_max_NAME_PAYMENT_TYPE_employer_last_1',
 'SK_ID_CURR_max_NAME_PRODUCT_TYPE_walk_in_last_1',
 'SK_ID_CURR_max_NAME_PRODUCT_TYPE_x_sell_last_1',
 'SK_ID_CURR_max_NFLAG_INSURED_ON_APPROVAL_last_1',
 'SK_ID_CURR_max_NUM_INSTALMENT_NUMBER_last_1',
 'SK_ID_CURR_max_NUM_INSTALMENT_NUMBER_last_5',
 'SK_ID_CURR_max_NUM_INSTALMENT_VERSION_last_1',
 'SK_ID_CURR_max_NUM_INSTALMENT_VERSION_last_5',
 'SK_ID_CURR_max_RATE_DOWN_PAYMENT_last_1',
 'SK_ID_CURR_max_SK_DPD',
 'SK_ID_CURR_max_SK_DPD_DEF',
 'SK_ID_CURR_max_SK_DPD_DEF_last_1',
 'SK_ID_CURR_max_SK_DPD_DEF_last_3',
 'SK_ID_CURR_max_SK_DPD_DEF_last_5',
 'SK_ID_CURR_max_SK_DPD_last_1',
 'SK_ID_CURR_max_SK_DPD_last_3',
 'SK_ID_CURR_max_SK_DPD_last_5',
 'SK_ID_CURR_max_application_credit_diff_last_1',
 'SK_ID_CURR_max_application_credit_ratio_last_1',
 'SK_ID_CURR_max_approved_last_1',
 'SK_ID_CURR_max_bureau_balance_dpd_level_last_1',
 'SK_ID_CURR_max_bureau_credit_type_car_last_1',
 'SK_ID_CURR_max_bureau_credit_type_consumer_last_1',
 'SK_ID_CURR_max_bureau_credit_type_credit_card_last_1',
 'SK_ID_CURR_max_bureau_credit_type_mortgage_last_1',
 'SK_ID_CURR_max_bureau_credit_type_other_last_1',
 'SK_ID_CURR_max_cash_loan_last_1',
 'SK_ID_CURR_max_cash_loan_last_5',
 'SK_ID_CURR_max_consumer_loan_last_1',
 'SK_ID_CURR_max_credit_goods_diff_last_1',
 'SK_ID_CURR_max_credit_goods_ratio_last_1',
 'SK_ID_CURR_max_instalment_paid_late_in_days_last_1',
 'SK_ID_CURR_max_instalment_paid_late_last_1',
 'SK_ID_CURR_max_instalment_paid_over_amount_last_1',
 'SK_ID_CURR_max_instalment_paid_over_amount_last_3',
 'SK_ID_CURR_max_instalment_paid_over_amount_last_5',
 'SK_ID_CURR_max_instalment_paid_over_last_1',
 'SK_ID_CURR_max_instalment_paid_over_last_3',
 'SK_ID_CURR_max_refused_last_1',
 'SK_ID_CURR_max_revolving_loan_last_1',
 'SK_ID_CURR_mean_AMT_CREDIT_MAX_OVERDUE_last_5',
 'SK_ID_CURR_mean_AMT_CREDIT_SUM_last_5',
 'SK_ID_CURR_mean_AMT_CREDIT_last_1',
 'SK_ID_CURR_mean_AMT_CREDIT_last_3',
 'SK_ID_CURR_mean_AMT_CREDIT_last_5',
 'SK_ID_CURR_mean_AMT_DOWN_PAYMENT_last_5',
 'SK_ID_CURR_mean_AMT_GOODS_PRICE_last_1',
 'SK_ID_CURR_mean_AMT_PAYMENT_last_1',
 'SK_ID_CURR_mean_AMT_PAYMENT_last_3',
 'SK_ID_CURR_mean_AMT_PAYMENT_last_5',
 'SK_ID_CURR_mean_DAYS_CREDIT_last_5',
 'SK_ID_CURR_mean_DAYS_DECISION_last_5',
 'SK_ID_CURR_mean_DAYS_ENTRY_PAYMENT_last_3',
 'SK_ID_CURR_mean_DAYS_ENTRY_PAYMENT_last_5',
 'SK_ID_CURR_mean_DAYS_INSTALMENT_last_1',
 'SK_ID_CURR_mean_DAYS_INSTALMENT_last_3',
 'SK_ID_CURR_mean_DAYS_INSTALMENT_last_5',
 'SK_ID_CURR_mean_HOUR_APPR_PROCESS_START_last_5',
 'SK_ID_CURR_mean_MONTHS_BALANCE_last_3',
 'SK_ID_CURR_mean_MONTHS_BALANCE_last_5',
 'SK_ID_CURR_mean_NAME_PAYMENT_TYPE_account_last_5',
 'SK_ID_CURR_mean_NUM_INSTALMENT_NUMBER_last_3',
 'SK_ID_CURR_mean_NUM_INSTALMENT_NUMBER_last_5',
 'SK_ID_CURR_mean_NUM_INSTALMENT_VERSION_last_5',
 'SK_ID_CURR_mean_SK_DPD',
 'SK_ID_CURR_mean_SK_DPD_DEF_last_1',
 'SK_ID_CURR_mean_SK_DPD_DEF_last_3',
 'SK_ID_CURR_mean_SK_DPD_DEF_last_5',
 'SK_ID_CURR_mean_SK_DPD_last_1',
 'SK_ID_CURR_mean_SK_DPD_last_3',
 'SK_ID_CURR_mean_SK_DPD_last_5',
 'SK_ID_CURR_mean_application_credit_diff_last_1',
 'SK_ID_CURR_mean_approved_last_1',
 'SK_ID_CURR_mean_bureau_balance_dpd_level_last_1',
 'SK_ID_CURR_mean_bureau_balance_dpd_level_last_3',
 'SK_ID_CURR_mean_bureau_balance_dpd_level_last_5',
 'SK_ID_CURR_mean_bureau_balance_status_unknown_last_1',
 'SK_ID_CURR_mean_bureau_balance_status_unknown_last_3',
 'SK_ID_CURR_mean_bureau_balance_status_unknown_last_5',
 'SK_ID_CURR_mean_bureau_credit_type_car_last_5',
 'SK_ID_CURR_mean_bureau_credit_type_consumer_last_5',
 'SK_ID_CURR_mean_bureau_credit_type_credit_card_last_5',
 'SK_ID_CURR_mean_bureau_credit_type_mortgage_last_5',
 'SK_ID_CURR_mean_bureau_credit_type_other_last_5',
 'SK_ID_CURR_mean_cash_loan_last_5',
 'SK_ID_CURR_mean_consumer_loan_last_5',
 'SK_ID_CURR_mean_refused_last_1',
 'SK_ID_CURR_median_SK_DPD_DEF_last_1',
 'SK_ID_CURR_median_SK_DPD_DEF_last_3',
 'SK_ID_CURR_median_SK_DPD_DEF_last_5',
 'SK_ID_CURR_median_SK_DPD_last_1',
 'SK_ID_CURR_median_SK_DPD_last_3',
 'SK_ID_CURR_median_SK_DPD_last_5',
 'SK_ID_CURR_min_AMT_ANNUITY_last_1_x',
 'SK_ID_CURR_min_AMT_ANNUITY_last_1_y',
 'SK_ID_CURR_min_AMT_ANNUITY_last_5_x',
 'SK_ID_CURR_min_AMT_APPLICATION_last_1',
 'SK_ID_CURR_min_AMT_CREDIT_MAX_OVERDUE_last_1',
 'SK_ID_CURR_min_AMT_CREDIT_MAX_OVERDUE_last_3',
 'SK_ID_CURR_min_AMT_CREDIT_MAX_OVERDUE_last_5',
 'SK_ID_CURR_min_AMT_CREDIT_SUM_DEBT_last_1',
 'SK_ID_CURR_min_AMT_CREDIT_SUM_LIMIT_last_1',
 'SK_ID_CURR_min_AMT_CREDIT_SUM_OVERDUE_last_1',
 'SK_ID_CURR_min_AMT_CREDIT_SUM_OVERDUE_last_5',
 'SK_ID_CURR_min_AMT_CREDIT_SUM_last_1',
 'SK_ID_CURR_min_AMT_CREDIT_SUM_last_5',
 'SK_ID_CURR_min_AMT_CREDIT_last_1',
 'SK_ID_CURR_min_AMT_CREDIT_last_3',
 'SK_ID_CURR_min_AMT_CREDIT_last_5',
 'SK_ID_CURR_min_AMT_DOWN_PAYMENT_last_1',
 'SK_ID_CURR_min_AMT_GOODS_PRICE_last_1',
 'SK_ID_CURR_min_AMT_INSTALMENT_last_1',
 'SK_ID_CURR_min_AMT_PAYMENT_last_1',
 'SK_ID_CURR_min_CNT_CREDIT_PROLONG_last_1',
 'SK_ID_CURR_min_CNT_CREDIT_PROLONG_last_5',
 'SK_ID_CURR_min_CNT_PAYMENT_last_1',
 'SK_ID_CURR_min_CREDIT_DAY_OVERDUE_last_1',
 'SK_ID_CURR_min_CREDIT_DAY_OVERDUE_last_5',
 'SK_ID_CURR_min_DAYS_CREDIT_ENDDATE_last_1',
 'SK_ID_CURR_min_DAYS_CREDIT_UPDATE_last_1',
 'SK_ID_CURR_min_DAYS_CREDIT_last_1',
 'SK_ID_CURR_min_DAYS_DECISION_last_1',
 'SK_ID_CURR_min_DAYS_ENTRY_PAYMENT_last_1',
 'SK_ID_CURR_min_DAYS_ENTRY_PAYMENT_last_3',
 'SK_ID_CURR_min_DAYS_ENTRY_PAYMENT_last_5',
 'SK_ID_CURR_min_DAYS_FIRST_DRAWING_last_1',
 'SK_ID_CURR_min_DAYS_INSTALMENT_last_1',
 'SK_ID_CURR_min_DAYS_INSTALMENT_last_3',
 'SK_ID_CURR_min_DAYS_INSTALMENT_last_5',
 'SK_ID_CURR_min_HOUR_APPR_PROCESS_START_last_1',
 'SK_ID_CURR_min_MONTHS_BALANCE',
 'SK_ID_CURR_min_MONTHS_BALANCE_last_1',
 'SK_ID_CURR_min_MONTHS_BALANCE_last_3',
 'SK_ID_CURR_min_MONTHS_BALANCE_last_5',
 'SK_ID_CURR_min_NAME_PAYMENT_TYPE_account_last_1',
 'SK_ID_CURR_min_NAME_PAYMENT_TYPE_account_last_5',
 'SK_ID_CURR_min_NAME_PAYMENT_TYPE_bank_last_1',
 'SK_ID_CURR_min_NAME_PAYMENT_TYPE_employer_last_1',
 'SK_ID_CURR_min_NAME_PAYMENT_TYPE_employer_last_5',
 'SK_ID_CURR_min_NAME_PRODUCT_TYPE_walk_in_last_1',
 'SK_ID_CURR_min_NAME_PRODUCT_TYPE_x_sell_last_1',
 'SK_ID_CURR_min_NFLAG_INSURED_ON_APPROVAL_last_1',
 'SK_ID_CURR_min_NUM_INSTALMENT_NUMBER_last_1',
 'SK_ID_CURR_min_NUM_INSTALMENT_NUMBER_last_5',
 'SK_ID_CURR_min_NUM_INSTALMENT_VERSION_last_1',
 'SK_ID_CURR_min_NUM_INSTALMENT_VERSION_last_5',
 'SK_ID_CURR_min_RATE_DOWN_PAYMENT_last_1',
 'SK_ID_CURR_min_SK_DPD_DEF_last_1',
 'SK_ID_CURR_min_SK_DPD_last_1',
 'SK_ID_CURR_min_application_credit_diff_last_1',
 'SK_ID_CURR_min_application_credit_ratio_last_1',
 'SK_ID_CURR_min_approved_last_1',
 'SK_ID_CURR_min_approved_last_5',
 'SK_ID_CURR_min_bureau_credit_type_car_last_1',
 'SK_ID_CURR_min_bureau_credit_type_car_last_5',
 'SK_ID_CURR_min_bureau_credit_type_consumer_last_1',
 'SK_ID_CURR_min_bureau_credit_type_credit_card_last_1',
 'SK_ID_CURR_min_bureau_credit_type_mortgage_last_1',
 'SK_ID_CURR_min_bureau_credit_type_mortgage_last_5',
 'SK_ID_CURR_min_bureau_credit_type_other_last_1',
 'SK_ID_CURR_min_cash_loan_last_1',
 'SK_ID_CURR_min_consumer_loan_last_1',
 'SK_ID_CURR_min_consumer_loan_last_5',
 'SK_ID_CURR_min_credit_goods_diff_last_1',
 'SK_ID_CURR_min_credit_goods_ratio_last_1',
 'SK_ID_CURR_min_instalment_paid_late_in_days_last_1',
 'SK_ID_CURR_min_instalment_paid_late_last_1',
 'SK_ID_CURR_min_instalment_paid_over_amount_last_1',
 'SK_ID_CURR_min_instalment_paid_over_last_1',
 'SK_ID_CURR_min_refused_last_1',
 'SK_ID_CURR_min_revolving_loan_last_1',
 'SK_ID_CURR_size_CREDIT_ACTIVE_last_3',
 'SK_ID_CURR_size_CREDIT_ACTIVE_last_5',
 'SK_ID_CURR_sum_AMT_ANNUITY_last_1_x',
 'SK_ID_CURR_sum_AMT_ANNUITY_last_1_y',
 'SK_ID_CURR_sum_AMT_ANNUITY_last_3_x',
 'SK_ID_CURR_sum_AMT_ANNUITY_last_5_x',
 'SK_ID_CURR_sum_AMT_APPLICATION_last_1',
 'SK_ID_CURR_sum_AMT_APPLICATION_last_3',
 'SK_ID_CURR_sum_AMT_CREDIT_MAX_OVERDUE_last_1',
 'SK_ID_CURR_sum_AMT_CREDIT_MAX_OVERDUE_last_3',
 'SK_ID_CURR_sum_AMT_CREDIT_MAX_OVERDUE_last_5',
 'SK_ID_CURR_sum_AMT_CREDIT_SUM_DEBT_last_1',
 'SK_ID_CURR_sum_AMT_CREDIT_SUM_DEBT_last_3',
 'SK_ID_CURR_sum_AMT_CREDIT_SUM_DEBT_last_5',
 'SK_ID_CURR_sum_AMT_CREDIT_SUM_LIMIT_last_1',
 'SK_ID_CURR_sum_AMT_CREDIT_SUM_LIMIT_last_3',
 'SK_ID_CURR_sum_AMT_CREDIT_SUM_LIMIT_last_5',
 'SK_ID_CURR_sum_AMT_CREDIT_SUM_OVERDUE_last_1',
 'SK_ID_CURR_sum_AMT_CREDIT_SUM_OVERDUE_last_3',
 'SK_ID_CURR_sum_AMT_CREDIT_SUM_OVERDUE_last_5',
 'SK_ID_CURR_sum_AMT_CREDIT_SUM_last_1',
 'SK_ID_CURR_sum_AMT_CREDIT_SUM_last_3',
 'SK_ID_CURR_sum_AMT_CREDIT_last_1',
 'SK_ID_CURR_sum_AMT_CREDIT_last_3',
 'SK_ID_CURR_sum_AMT_CREDIT_last_5',
 'SK_ID_CURR_sum_AMT_DOWN_PAYMENT_last_1',
 'SK_ID_CURR_sum_AMT_DOWN_PAYMENT_last_3',
 'SK_ID_CURR_sum_AMT_DRAWINGS_ATM_CURRENT',
 'SK_ID_CURR_sum_AMT_DRAWINGS_CURRENT',
 'SK_ID_CURR_sum_AMT_GOODS_PRICE_last_1',
 'SK_ID_CURR_sum_AMT_GOODS_PRICE_last_3',
 'SK_ID_CURR_sum_AMT_GOODS_PRICE_last_5',
 'SK_ID_CURR_sum_AMT_INSTALMENT_last_1',
 'SK_ID_CURR_sum_AMT_INSTALMENT_last_3',
 'SK_ID_CURR_sum_AMT_INSTALMENT_last_5',
 'SK_ID_CURR_sum_AMT_PAYMENT_last_1',
 'SK_ID_CURR_sum_AMT_PAYMENT_last_3',
 'SK_ID_CURR_sum_AMT_PAYMENT_last_5',
 'SK_ID_CURR_sum_CNT_CREDIT_PROLONG_last_1',
 'SK_ID_CURR_sum_CNT_CREDIT_PROLONG_last_3',
 'SK_ID_CURR_sum_CNT_CREDIT_PROLONG_last_5',
 'SK_ID_CURR_sum_CNT_INSTALMENT_MATURE_CUM',
 'SK_ID_CURR_sum_CNT_PAYMENT_last_1',
 'SK_ID_CURR_sum_CREDIT_DAY_OVERDUE_last_1',
 'SK_ID_CURR_sum_CREDIT_DAY_OVERDUE_last_3',
 'SK_ID_CURR_sum_CREDIT_DAY_OVERDUE_last_5',
 'SK_ID_CURR_sum_DAYS_CREDIT_ENDDATE_last_1',
 'SK_ID_CURR_sum_DAYS_CREDIT_ENDDATE_last_3',
 'SK_ID_CURR_sum_DAYS_CREDIT_UPDATE_last_1',
 'SK_ID_CURR_sum_DAYS_CREDIT_last_1',
 'SK_ID_CURR_sum_DAYS_DECISION_last_1',
 'SK_ID_CURR_sum_DAYS_ENTRY_PAYMENT_last_1',
 'SK_ID_CURR_sum_DAYS_ENTRY_PAYMENT_last_3',
 'SK_ID_CURR_sum_DAYS_ENTRY_PAYMENT_last_5',
 'SK_ID_CURR_sum_DAYS_FIRST_DRAWING_last_1',
 'SK_ID_CURR_sum_DAYS_INSTALMENT_last_1',
 'SK_ID_CURR_sum_DAYS_INSTALMENT_last_3',
 'SK_ID_CURR_sum_DAYS_INSTALMENT_last_5',
 'SK_ID_CURR_sum_HOUR_APPR_PROCESS_START_last_1',
 'SK_ID_CURR_sum_MONTHS_BALANCE',
 'SK_ID_CURR_sum_MONTHS_BALANCE_last_1',
 'SK_ID_CURR_sum_MONTHS_BALANCE_last_3',
 'SK_ID_CURR_sum_MONTHS_BALANCE_last_5',
 'SK_ID_CURR_sum_NAME_PAYMENT_TYPE_account_last_1',
 'SK_ID_CURR_sum_NAME_PAYMENT_TYPE_account_last_3',
 'SK_ID_CURR_sum_NAME_PAYMENT_TYPE_bank_last_1',
 'SK_ID_CURR_sum_NAME_PAYMENT_TYPE_employer_last_1',
 'SK_ID_CURR_sum_NAME_PAYMENT_TYPE_employer_last_3',
 'SK_ID_CURR_sum_NAME_PAYMENT_TYPE_employer_last_5',
 'SK_ID_CURR_sum_NAME_PRODUCT_TYPE_walk_in_last_1',
 'SK_ID_CURR_sum_NAME_PRODUCT_TYPE_x_sell_last_1',
 'SK_ID_CURR_sum_NAME_PRODUCT_TYPE_x_sell_last_3',
 'SK_ID_CURR_sum_NFLAG_INSURED_ON_APPROVAL_last_1',
 'SK_ID_CURR_sum_NUM_INSTALMENT_NUMBER_last_1',
 'SK_ID_CURR_sum_NUM_INSTALMENT_NUMBER_last_3',
 'SK_ID_CURR_sum_NUM_INSTALMENT_NUMBER_last_5',
 'SK_ID_CURR_sum_NUM_INSTALMENT_VERSION_last_1',
 'SK_ID_CURR_sum_NUM_INSTALMENT_VERSION_last_3',
 'SK_ID_CURR_sum_NUM_INSTALMENT_VERSION_last_5',
 'SK_ID_CURR_sum_RATE_DOWN_PAYMENT_last_1',
 'SK_ID_CURR_sum_SK_DPD',
 'SK_ID_CURR_sum_SK_DPD_DEF',
 'SK_ID_CURR_sum_SK_DPD_DEF_last_5',
 'SK_ID_CURR_sum_SK_DPD_last_3',
 'SK_ID_CURR_sum_SK_DPD_last_5',
 'SK_ID_CURR_sum_application_credit_diff_last_1',
 'SK_ID_CURR_sum_application_credit_diff_last_3',
 'SK_ID_CURR_sum_application_credit_ratio_last_1',
 'SK_ID_CURR_sum_approved_last_1',
 'SK_ID_CURR_sum_bureau_balance_dpd_level_last_5',
 'SK_ID_CURR_sum_bureau_credit_type_car_last_1',
 'SK_ID_CURR_sum_bureau_credit_type_car_last_3',
 'SK_ID_CURR_sum_bureau_credit_type_consumer_last_1',
 'SK_ID_CURR_sum_bureau_credit_type_credit_card_last_1',
 'SK_ID_CURR_sum_bureau_credit_type_mortgage_last_1',
 'SK_ID_CURR_sum_bureau_credit_type_mortgage_last_3',
 'SK_ID_CURR_sum_bureau_credit_type_mortgage_last_5',
 'SK_ID_CURR_sum_bureau_credit_type_other_last_1',
 'SK_ID_CURR_sum_bureau_credit_type_other_last_3',
 'SK_ID_CURR_sum_cash_loan_last_1',
 'SK_ID_CURR_sum_cash_loan_last_3',
 'SK_ID_CURR_sum_consumer_loan_last_1',
 'SK_ID_CURR_sum_credit_goods_diff_last_1',
 'SK_ID_CURR_sum_credit_goods_ratio_last_1',
 'SK_ID_CURR_sum_instalment_paid_late_in_days_last_1',
 'SK_ID_CURR_sum_instalment_paid_late_in_days_last_3',
 'SK_ID_CURR_sum_instalment_paid_late_in_days_last_5',
 'SK_ID_CURR_sum_instalment_paid_late_last_1',
 'SK_ID_CURR_sum_instalment_paid_late_last_3',
 'SK_ID_CURR_sum_instalment_paid_late_last_5',
 'SK_ID_CURR_sum_instalment_paid_over_amount_last_1',
 'SK_ID_CURR_sum_instalment_paid_over_amount_last_3',
 'SK_ID_CURR_sum_instalment_paid_over_amount_last_5',
 'SK_ID_CURR_sum_instalment_paid_over_last_1',
 'SK_ID_CURR_sum_instalment_paid_over_last_3',
 'SK_ID_CURR_sum_instalment_paid_over_last_5',
 'SK_ID_CURR_sum_refused_last_1',
 'SK_ID_CURR_sum_refused_last_3',
 'SK_ID_CURR_sum_refused_last_5',
 'SK_ID_CURR_sum_revolving_loan_last_1',
 'SK_ID_CURR_sum_revolving_loan_last_3',
 'SK_ID_CURR_var_AMT_ANNUITY_last_5_x',
 'SK_ID_CURR_var_AMT_CREDIT_MAX_OVERDUE_last_3',
 'SK_ID_CURR_var_AMT_CREDIT_MAX_OVERDUE_last_5',
 'SK_ID_CURR_var_AMT_CREDIT_SUM_last_5',
 'SK_ID_CURR_var_AMT_CREDIT_last_3',
 'SK_ID_CURR_var_AMT_CREDIT_last_5',
 'SK_ID_CURR_var_AMT_DOWN_PAYMENT_last_5',
 'SK_ID_CURR_var_AMT_PAYMENT_last_3',
 'SK_ID_CURR_var_AMT_PAYMENT_last_5',
 'SK_ID_CURR_var_DAYS_FIRST_DRAWING_last_3',
 'SK_ID_CURR_var_DAYS_FIRST_DRAWING_last_5',
 'SK_ID_CURR_var_DAYS_INSTALMENT_last_3',
 'SK_ID_CURR_var_DAYS_INSTALMENT_last_5',
 'SK_ID_CURR_var_MONTHS_BALANCE',
 'SK_ID_CURR_var_NAME_PAYMENT_TYPE_account_last_3',
 'SK_ID_CURR_var_NAME_PAYMENT_TYPE_employer_last_3',
 'SK_ID_CURR_var_NAME_PAYMENT_TYPE_employer_last_5',
 'SK_ID_CURR_var_NUM_INSTALMENT_VERSION_last_5',
 'SK_ID_CURR_var_SK_DPD',
 'SK_ID_CURR_var_SK_DPD_DEF',
 'SK_ID_CURR_var_bureau_credit_type_car_last_3',
 'SK_ID_CURR_var_bureau_credit_type_mortgage_last_3',
 'SK_ID_CURR_var_bureau_credit_type_mortgage_last_5',
 'SK_ID_CURR_var_credit_goods_diff_last_3',
 'SK_ID_CURR_var_credit_goods_diff_last_5',
 'SK_ID_CURR_var_instalment_paid_late_last_3',
 'SK_ID_CURR_var_instalment_paid_late_last_5',
 'SK_ID_CURR_var_instalment_paid_over_last_3',
 'SK_ID_CURR_var_instalment_paid_over_last_5',
 'application_previous_application_annuity_diff_last_10_applications_mean',
 'application_previous_application_annuity_diff_last_5_applications_mean',
 'application_previous_application_annuity_ratio_last_10_applications_mean',
 'application_previous_application_credit_diff_last_10_applications_mean',
 'application_previous_application_credit_diff_last_5_applications_mean',
 'application_previous_application_credit_ratio_last_10_applications_mean',
 'application_previous_application_credit_ratio_last_5_applications_mean',
 'application_previous_application_hour_diff_last_10_applications_mean',
 'application_previous_application_hour_diff_last_5_applications_mean',
 'application_previous_application_the_same_contract_type_last_10_applications_mean',
 'application_previous_application_the_same_contract_type_last_5_applications_mean',
 'application_previous_application_the_same_type_suite_last_10_applications_mean',
 'application_previous_application_the_same_type_suite_last_5_applications_mean',
 'application_previous_application_the_same_weekday_last_10_applications_mean',
 'exc_tf_1','exc_tf_2', 'exc_tf_3'}

In [6]:
# 得到最终的数据集，全部是数值型数据
df_reduced = application_train.drop(columns=to_drop)

In [9]:
df_reduced.to_parquet('/Users/xiaohehe/Desktop/hkust/24-25fall/fintech/project1/final_data/combine_data_drop.parquet')

In [19]:
df_reduced

,NAME_CONTRACT_TYPE,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,...,SK_ID_CURR_var_refused_last_5,SK_ID_CURR_var_revolving_loan_last_5,SK_ID_CURR_var_cash_loan_last_5,SK_ID_CURR_var_consumer_loan_last_5,SK_ID_CURR_var_credit_goods_ratio_last_5,SK_ID_CURR_var_application_credit_diff_last_5,SK_ID_CURR_var_application_credit_ratio_last_5,previous_application_number_of_prev_application,previous_application_fraction_of_refused_applications,TARGET
0,0,0.0,1.0,0.0,0,1,0.0,0.0,0.0,0.0,...,0.100186,0.097330,0.167730,0.171049,0.017306,4.953836e+09,0.069064,1.0,0.000000,1
1,0,1.0,0.0,0.0,0,0,0.0,1.0,0.0,0.0,...,0.000000,0.000000,0.333333,0.333333,0.007015,5.728765e+09,0.005324,3.0,0.000000,0
2,1,0.0,1.0,0.0,1,1,0.0,0.0,0.0,0.0,...,0.100186,0.097330,0.167730,0.171049,0.017306,4.953836e+09,0.069064,1.0,0.000000,0
3,0,1.0,0.0,0.0,0,1,0.0,0.0,0.0,0.0,...,0.200000,0.300000,0.300000,0.000000,0.050180,9.514831e+09,0.028940,9.0,0.111111,0
4,0,0.0,1.0,0.0,0,1,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.200000,0.200000,0.012347,6.140349e+08,0.007897,6.0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,0,1.0,0.0,0.0,0,1,0.0,0.0,0.0,0.0,...,0.105144,0.106273,0.166300,0.170487,0.016400,4.470164e+09,0.065782,1.0,0.000000,2
48740,0,1.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.250000,0.250000,0.012453,1.640626e+08,0.009809,4.0,0.000000,2
48741,0,1.0,0.0,0.0,1,1,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.005002,1.459742e+08,0.006176,2.0,0.000000,2
48742,0,0.0,1.0,0.0,0,0,0.0,1.0,0.0,0.0,...,0.300000,0.000000,0.300000,0.300000,0.010553,5.010032e+08,0.007871,5.0,0.400000,2


In [20]:
train_data = df_reduced[df_reduced['TARGET']!=2]
test_data = df_reduced[df_reduced['TARGET']==2].drop('TARGET',axis =1)
del application_train

NameError: name 'application_train' is not defined

In [22]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import lightgbm as lgb



# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(train_data.drop('TARGET',axis =1 ), train_data['TARGET'], test_size=0.2, random_state=42)


# 创建XGBoost模型
model = lgb.LGBMClassifier(use_label_encoder=False, eval_metric='logloss')

# 拟合模型
model.fit(X_train, y_train)
          #eval_set=[(validation_data.drop('TARGET',axis =1), validation_data['TARGET'])],verbose = TRUE  # 验证集)



[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: use_label_encoder


[LightGBM] [Fatal] Do not support special JSON characters in feature name.


LightGBMError: Do not support special JSON characters in feature name.

In [15]:
# 计算指标

from sklearn.metrics import roc_auc_score
#test_encoded = pd.get_dummies(application_test, columns=columns, drop_first=True)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # 获取正类的概率

# 计算 AUC
auc = roc_auc_score(y_test, y_pred_proba)

print(f'AUC: {auc}')


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_pred = model.predict(X_test)

# 计算各项指标
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# 输出结果
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

AUC: 0.9803355715011737
Accuracy: 0.9557638735352642
Precision: 0.9934698098393305
Recall: 0.9175613293478838
F1 Score: 0.9540079810220857


In [16]:
test_ori = pd.read_csv('/Users/xiaohehe/Desktop/hkust/24-25fall/fintech/project1/data/application_test.csv')
test_ori

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,456221,Cash loans,F,N,Y,0,121500.0,412560.0,17473.5,270000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
48740,456222,Cash loans,F,N,N,2,157500.0,622413.0,31909.5,495000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
48741,456223,Cash loans,F,Y,Y,1,202500.0,315000.0,33205.5,315000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,3.0,1.0
48742,456224,Cash loans,M,N,N,0,225000.0,450000.0,25128.0,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0


In [17]:
y_pred_proba = model.predict_proba(test_data)[:, 1]
y_pred_proba = pd.DataFrame(y_pred_proba, columns=['TARGET'],index = test_ori['SK_ID_CURR'])
y_pred_proba.index.name = 'SK_ID_CURR'
y_pred_proba

,TARGET
SK_ID_CURR,
100001,0.499028
100005,0.940297
100013,0.886752
100028,0.102738
100038,0.941180
...,...
456221,0.834482
456222,0.857370
456223,0.016814


In [20]:
y_pred_proba.to_csv('/Users/xiaohehe/Desktop/hkust/24-25fall/fintech/project1/result/y_pred_only_drop.csv')

In [14]:
importance = model.feature_importances_

# 创建特征重要性数据框
feature_importance = pd.DataFrame({
    'Feature': train_data.drop('TARGET',axis=1).columns,
    'Importance': importance
}).sort_values(by='Importance', ascending=False)

# 打印特征重要性
print(feature_importance)

                                                Feature  Importance
249      CODE_GENDER_NAME_EDUCATION_TYPE_max_AMT_CREDIT    0.037821
876          SK_ID_CURR_min_instalment_paid_over_last_5    0.031758
526   SK_ID_CURR_mean_bureau_credit_type_credit_card...    0.025303
300                                     long_employment    0.024028
283                   OCCUPATION_TYPE_mean_EXT_SOURCE_2    0.023252
...                                                 ...         ...
445                 SK_ID_CURR_count_CREDIT_TYPE_last_1    0.000000
920                     SK_ID_CURR_median_SK_DPD_last_1    0.000000
919                        SK_ID_CURR_min_SK_DPD_last_1    0.000000
918                        SK_ID_CURR_max_SK_DPD_last_1    0.000000
1318                                           exc_tf_3    0.000000

[1319 rows x 2 columns]


In [15]:
# 筛选指标

feature_importance['cumulative_importance'] = feature_importance['Importance'].cumsum()

# 选择累计重要性达到80%的因子
selected_features = feature_importance[feature_importance['cumulative_importance'] <= 0.8]

# 获取选择的因子
result = selected_features['Feature']

result

249        CODE_GENDER_NAME_EDUCATION_TYPE_max_AMT_CREDIT
876            SK_ID_CURR_min_instalment_paid_over_last_5
526     SK_ID_CURR_mean_bureau_credit_type_credit_card...
300                                       long_employment
283                     OCCUPATION_TYPE_mean_EXT_SOURCE_2
                              ...                        
347     NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_OWN_C...
752                             SK_ID_CURR_var_SK_DPD_DEF
606     SK_ID_CURR_mean_bureau_credit_type_consumer_la...
1222              SK_ID_CURR_min_RATE_DOWN_PAYMENT_last_5
947     SK_ID_CURR_mean_pos_cash_paid_late_with_tolera...
Name: Feature, Length: 91, dtype: object

In [16]:
trian_data_flit = train_data[result]
test_data_flit = test_data[result]

In [17]:
trian_data_flit

,CODE_GENDER_NAME_EDUCATION_TYPE_max_AMT_CREDIT,SK_ID_CURR_min_instalment_paid_over_last_5,SK_ID_CURR_mean_bureau_credit_type_credit_card_last_3,long_employment,OCCUPATION_TYPE_mean_EXT_SOURCE_2,SK_ID_CURR_sum_NFLAG_INSURED_ON_APPROVAL_last_3,REG_CITY_NOT_WORK_CITY,retirement_age,CODE_GENDER_REG_CITY_NOT_WORK_CITY_mean_CNT_CHILDREN_abs_diff,WALLSMATERIAL_MODE_XNA,...,FLAG_DOCUMENT_6,SK_ID_CURR_max_refused_last_3,NAME_INCOME_TYPE_Working,SK_ID_CURR_var_NFLAG_INSURED_ON_APPROVAL_last_5,SK_ID_CURR_max_NAME_PRODUCT_TYPE_walk_in_last_3,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_OWN_CAR_AGE_abs_diff,SK_ID_CURR_var_SK_DPD_DEF,SK_ID_CURR_mean_bureau_credit_type_consumer_last_5,SK_ID_CURR_min_RATE_DOWN_PAYMENT_last_5,SK_ID_CURR_mean_pos_cash_paid_late_with_tolerance_last_5
0,-1.343121,-0.035233,1.538042,-0.538748,-0.722072,-0.728461,-0.450107,-0.951505,-0.371169,-1.196479,...,-0.231431,-0.689145,0.802396,-0.009014,-0.541237,-0.008780,-0.003195,-1.180539,-0.701221,-0.217383
1,0.787838,-0.035233,1.538042,-0.538748,0.689750,2.708361,-0.450107,1.050967,-0.560519,-1.196479,...,-0.231431,-0.689145,-1.246268,0.952288,-0.541237,-0.008780,-0.003195,-0.781044,-0.701221,-0.217383
2,-1.343121,-0.035233,-1.068692,-0.538748,-0.722072,-0.728461,-0.450107,1.050967,-0.371169,0.835786,...,-0.231431,-0.689145,0.802396,-0.009014,-0.541237,1.200224,-0.003195,1.216433,2.346126,-0.217383
3,0.744162,-0.035233,-0.035554,1.856154,-0.722072,-0.728461,-0.450107,1.050967,-0.560519,0.835786,...,-0.231431,1.724077,0.802396,-0.009014,-0.541237,-0.008780,-0.018079,0.035422,0.024063,-0.217383
4,-1.343121,-0.035233,-1.068692,1.856154,0.689750,0.989950,2.221692,1.050967,-0.193100,0.835786,...,-0.231431,-0.689145,0.802396,0.952288,2.247892,-0.008780,-0.003195,1.216433,0.738210,-0.217383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565367,-0.925099,-0.035233,2.265238,-0.538748,-0.751583,-0.728461,-0.450107,-0.951505,1.863487,0.835786,...,-0.231431,-0.689145,0.802396,-0.009014,-0.541237,-0.207453,-0.003195,-2.214970,-0.701221,2.139062
565368,0.744162,-0.035233,0.234675,-0.538748,-0.628307,-0.728461,-0.450107,-0.951505,1.803243,0.835786,...,-0.231431,-0.689145,0.802396,-0.009014,-0.541237,-0.069775,-0.003195,-0.381548,0.645705,-0.217383
565369,0.744162,-0.035233,0.830055,-0.538748,1.956429,-0.728461,-0.450107,-0.951505,-0.095598,0.835786,...,-0.231431,1.724077,0.802396,-0.009014,-0.541237,-0.008780,-0.007674,-0.811226,0.024063,-0.217383
565370,0.764081,-0.035233,0.234675,1.856154,1.329731,-0.728461,-0.450107,1.050967,-0.560519,0.835786,...,-0.231431,1.724077,0.802396,1.173083,2.247892,-0.008780,-0.010900,-0.381548,0.024063,-0.217383


In [18]:
# 重新训练


# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(trian_data_flit, train_data['TARGET'], test_size=0.2, random_state=42)\


# 创建XGBoost模型
model_flit = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# 拟合模型
model_flit.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [19]:
# 计算指标

from sklearn.metrics import roc_auc_score
#test_encoded = pd.get_dummies(application_test, columns=columns, drop_first=True)
y_pred_proba = model_flit.predict_proba(X_test)[:, 1]  # 获取正类的概率

# 计算 AUC
auc = roc_auc_score(y_test, y_pred_proba)

print(f'AUC: {auc}')


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_pred = model_flit.predict(X_test)

# 计算各项指标
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# 输出结果
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

AUC: 0.9785766692591029
Accuracy: 0.9553393765200089
Precision: 0.995343736170704
Recall: 0.9149613541095527
F1 Score: 0.9534613683279268


In [20]:
test_data_flit = test_data[result]
test_data_flit

,CODE_GENDER_NAME_EDUCATION_TYPE_max_AMT_CREDIT,SK_ID_CURR_min_instalment_paid_over_last_5,SK_ID_CURR_mean_bureau_credit_type_credit_card_last_3,long_employment,OCCUPATION_TYPE_mean_EXT_SOURCE_2,SK_ID_CURR_sum_NFLAG_INSURED_ON_APPROVAL_last_3,REG_CITY_NOT_WORK_CITY,retirement_age,CODE_GENDER_REG_CITY_NOT_WORK_CITY_mean_CNT_CHILDREN_abs_diff,WALLSMATERIAL_MODE_XNA,...,FLAG_DOCUMENT_6,SK_ID_CURR_max_refused_last_3,NAME_INCOME_TYPE_Working,SK_ID_CURR_var_NFLAG_INSURED_ON_APPROVAL_last_5,SK_ID_CURR_max_NAME_PRODUCT_TYPE_walk_in_last_3,NAME_EDUCATION_TYPE_OCCUPATION_TYPE_mean_OWN_CAR_AGE_abs_diff,SK_ID_CURR_var_SK_DPD_DEF,SK_ID_CURR_mean_bureau_credit_type_consumer_last_5,SK_ID_CURR_min_RATE_DOWN_PAYMENT_last_5,SK_ID_CURR_mean_pos_cash_paid_late_with_tolerance_last_5
0,-2.743213,-0.035233,-1.068692,1.856154,0.202593,-0.728461,-0.450107,1.050967,-0.600615,-1.196479,...,-0.231431,-0.689145,0.802396,0.067293,-0.541237,-0.096373,0.015218,1.216433,0.798339,-0.217383
1,-2.917564,-0.035233,0.234675,1.856154,-3.504653,-0.728461,-0.450107,1.050967,-0.400153,0.835786,...,-0.231431,-0.689145,0.802396,0.067293,-0.541237,-0.096373,0.015218,-0.115218,0.865002,-0.217383
2,-2.917564,-0.035233,-1.068692,1.856154,-0.037561,0.989950,-0.450107,1.050967,-0.400153,0.835786,...,-0.231431,-0.689145,0.802396,0.952288,-0.541237,-0.366815,-0.018077,-0.781044,-0.701221,-0.217383
3,-2.910520,-0.035233,1.538042,-0.538748,-0.792326,-0.728461,-0.450107,-0.951505,2.621044,-1.196479,...,-0.231431,-0.689145,0.802396,-1.471661,-0.541237,-0.096373,-0.018079,-1.180539,-0.701221,-0.217383
4,-2.917564,-0.035233,-0.057782,1.856154,0.202593,-0.728461,2.221692,-0.951505,-0.222459,0.835786,...,-0.231431,-0.689145,0.802396,0.067293,-0.541237,-0.096373,0.015218,0.022489,0.557254,-0.217383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,-2.910520,-0.035233,-1.068692,1.856154,0.202593,0.989950,-0.450107,1.050967,-0.600615,0.835786,...,-0.231431,-0.689145,0.802396,0.067293,2.247892,-0.096373,0.015218,-0.381548,-0.003622,-0.217383
48740,-2.910520,-0.035233,-0.057782,-0.538748,-0.792326,0.989950,2.221692,-0.951505,2.340355,0.835786,...,-0.231431,-0.689145,-1.246268,0.952288,-0.541237,-0.096373,0.015218,0.022489,-0.701221,-0.217383
48741,-2.910520,-0.035233,0.234675,1.856154,0.202593,-0.728461,-0.450107,1.050967,0.144583,-1.196479,...,-0.231431,-0.689145,-1.246268,-1.471661,-0.541237,-0.096373,0.015218,-0.381548,-0.701221,-0.217383
48742,-2.917564,-0.035233,0.234675,1.856154,2.190201,-0.728461,2.221692,-0.951505,-0.233573,-1.196479,...,-0.231431,1.724077,-1.246268,-1.471661,-0.541237,-0.096373,0.015218,0.417442,-0.701221,-0.217383


In [21]:
y_pred_proba = model_flit.predict_proba(test_data_flit)[:, 1]
y_pred_proba = pd.DataFrame(y_pred_proba, columns=['TARGET'],index = test_ori['SK_ID_CURR'])
y_pred_proba.index.name = 'SK_ID_CURR'
y_pred_proba

,TARGET
SK_ID_CURR,
100001,0.648629
100005,0.751071
100013,0.937631
100028,0.098483
100038,0.951474
...,...
456221,0.853421
456222,0.976353
456223,0.028339


In [22]:
y_pred_proba.to_csv('/Users/xiaohehe/Desktop/hkust/24-25fall/fintech/project1/result/y_pred_stand_embeded.csv')

In [86]:
model.save_model('/home/cheam/fin_project1/fintech_pro1_data/xgb_model_1.json')
#model_flit.save_model('/home/cheam/fin_project1/fintech_pro1_data/xgb_model_flit_1.json')

In [24]:
application_train

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,SK_ID_CURR_var_credit_goods_diff_last_5,SK_ID_CURR_var_credit_goods_ratio_last_5,SK_ID_CURR_var_application_credit_diff_last_5,SK_ID_CURR_var_application_credit_ratio_last_5,previous_application_number_of_prev_application,previous_application_fraction_of_refused_applications,exc_tf_1,exc_tf_2,exc_tf_3,TARGET
0,100002,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,...,1.785162e+09,0.017306,4.953836e+09,0.069064,1.0,0.000000,0,0,0,1
1,100003,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,...,5.728765e+09,0.007015,5.728765e+09,0.005324,3.0,0.000000,0,0,0,0
2,100004,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,...,1.785162e+09,0.017306,4.953836e+09,0.069064,1.0,0.000000,0,0,0,0
3,100006,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,...,2.378708e+10,0.050180,9.514831e+09,0.028940,9.0,0.111111,0,0,0,0
4,100007,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,...,6.140349e+08,0.012347,6.140349e+08,0.007897,6.0,0.000000,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,456221,Cash loans,F,N,Y,0,121500.0,412560.0,17473.5,270000.0,...,1.876021e+09,0.016400,4.470164e+09,0.065782,1.0,0.000000,0,0,0,2
48740,456222,Cash loans,F,N,N,2,157500.0,622413.0,31909.5,495000.0,...,1.640626e+08,0.012453,1.640626e+08,0.009809,4.0,0.000000,0,0,0,2
48741,456223,Cash loans,F,Y,Y,1,202500.0,315000.0,33205.5,315000.0,...,1.459742e+08,0.005002,1.459742e+08,0.006176,2.0,0.000000,0,0,0,2
48742,456224,Cash loans,M,N,N,0,225000.0,450000.0,25128.0,450000.0,...,5.884195e+08,0.010553,5.010032e+08,0.007871,5.0,0.400000,0,0,0,2
